In [1]:
import gensim.downloader as api
w2v_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
from gensim.similarities import Similarity

In [2]:
import numpy as np
import pandas as pd
import re

In [14]:
# Load the dataset
synonym_test_dataset = pd.read_csv('A2-DataSet/synonym.csv')

print(synonym_test_dataset.head())
print(synonym_test_dataset.info())

      question        answer              0               1              2  \
0   enormously  tremendously  appropriately        uniquely   tremendously   
1   provisions  stipulations   stipulations  interrelations  jurisdictions   
2  haphazardly      randomly    dangerously         densely       randomly   
3    prominent   conspicuous       battered         ancient     mysterious   
4       zenith      pinnacle     completion        pinnacle         outset   

                 3  
0        decidedly  
1  interpretations  
2         linearly  
3      conspicuous  
4          decline  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  80 non-null     object
 1   answer    80 non-null     object
 2   0         80 non-null     object
 3   1         80 non-null     object
 4   2         80 non-null     object
 5   3         80 non-null     object

In [29]:
def closest_synonym(query, list_of_guess_words):
    
    closest_synonym = None
    
    # iterate through the list of guess words and find the guess word with the highest similarity score
    for guess_word in list_of_guess_words:
        sim_score = w2v_model.similarity(query, guess_word)
        
        if closest_synonym is None:
            closest_synonym = (guess_word, sim_score)
        elif sim_score > closest_synonym[1]:
            closest_synonym = (guess_word, sim_score)
    
    return closest_synonym[0]

# print(closest_synonym('cat', ['dog', 'mouse', 'house', 'tree']))

In [28]:
def assign_label(question_word, correct_answer, closest_synonym, guess_words):
    # Check if question word and at least one guess word are in the vocabulary
    if question_word in w2v_model.vocab and any(word in w2v_model.vocab for word in guess_words):
        # Check if the closest guess is correct
        if closest_synonym is not None and closest_synonym == correct_answer:
            return "correct"
        else:
            return "wrong"
    else:
        return "guess"

In [35]:
# Function to process the CSV file and apply the closest_synonym function
def process_csv(file_path):
    
    question_words = []
    answer_words = []
    guess_words = []
    labels = []
    
    
    # Read the CSV file into a Pandas DataFrame, skipping the first row
    synonym_test_dataset = pd.read_csv(file_path)

    # Process each row in the DataFrame
    for index, row in synonym_test_dataset.iterrows():
        # Split the row into words based on comma
        words = row[0]
        print(words)

        # Store the first word in 'query' and the rest in 'list_of_guess_words'
        query = words[0]
        answer = words[1]
        list_of_guess_words = words[2:]

        # Call the 'closest_synonym' function and store the result
        result = closest_synonym(query, list_of_guess_words)
        
        question_words.append(query)
        answer_words.append(answer)
        guess_words.append(result)
        labels.append(assign_label(query, answer, result, list_of_guess_words))
    
    
    results_df = pd.DataFrame({'question_word': question_words, 'answer_word': answer_words, 'guess_word': guess_words, 'label': labels})    
    results_df.to_csv('word2vec-google-news-300-details.csv', index=False)
    
    return results_df

file_path = 'A2-DataSet/synonym.csv' 
processed_results = process_csv(file_path)
processed_results

/var/folders/2m/tvft4g1n2t7bhn7xdc4t2l2m0000gn/T/ipykernel_97171/3421812679.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  words = row[0]


enormously


AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4